In [112]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import matplotlib.pyplot as plt
os.chdir("/Users/burke/Documents/research/bpCog/microsim")
from microsim.population import NHANESDirectSamplePopulation
from microsim.outcome import OutcomeType

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

import multiprocessing as mp
#mp.set_start_method('forkserver')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
pop = NHANESDirectSamplePopulation(150000, 1999)
pop.advance_multi_process(20)

processing year: 0
processing year: 1
processing year: 2
processing year: 3
processing year: 4
processing year: 5
processing year: 6
processing year: 7
processing year: 8
processing year: 9
processing year: 10
processing year: 11
processing year: 12
processing year: 13
processing year: 14
processing year: 15
processing year: 16
processing year: 17
processing year: 18
processing year: 19


In [73]:
df = pop.get_people_current_state_as_dataframe()

In [74]:
df['incidentDementia'] = [person.has_outcome_during_simulation(OutcomeType.DEMENTIA) for i, person in pop._people.iteritems()]
df['ageAtFirstMI'] = [person.get_age_at_first_outcome_in_sim(OutcomeType.MI) for i, person in pop._people.iteritems()]
df['ageAtFirstStroke'] = [person.get_age_at_first_outcome_in_sim(OutcomeType.STROKE) for i, person in pop._people.iteritems()]
df['ageAtFirstDementia'] = [person.get_age_at_first_outcome_in_sim(OutcomeType.DEMENTIA) for i, person in pop._people.iteritems()]
df['waveOfFirstMI'] = df.ageAtFirstMI - df.baseAge + 1
df['waveOfFirstStroke'] = df.ageAtFirstStroke - df.baseAge + 1
df['waveOfFirstDementia'] = df.ageAtFirstDementia - df.baseAge + 1

df['waveOfFirstMI']= df['waveOfFirstMI'].fillna(1)
df['waveOfFirstStroke']= df['waveOfFirstStroke'].fillna(1)
df['waveOfFirstDementia']= df['waveOfFirstDementia'].fillna(1)


df['waveOfFirstMI']= df['waveOfFirstMI'].astype('int')
df['waveOfFirstStroke']= df['waveOfFirstStroke'].astype('int')
df['waveOfFirstDementia']= df['waveOfFirstDementia'].astype('int')

In [128]:
df['qalysPostMI'] = [pop._people.iloc[index].get_qalys_from_wave(dfPerson.waveOfFirstMI) for index, dfPerson in df.iterrows()]
df['qalysPostStroke'] = [pop._people.iloc[index].get_qalys_from_wave(dfPerson.waveOfFirstStroke) for index, dfPerson in df.iterrows()]
df['qalysPostDementia'] = [pop._people.iloc[index].get_qalys_from_wave(dfPerson.waveOfFirstDementia) for index, dfPerson in df.iterrows()]

In [76]:
df.groupby('miInSim')[['qalysPostMI','age']].describe()

qalysPostMI                                                          \
              count       mean       std       min    25%      50%      75%   
miInSim                                                                       
False      139953.0  18.851588  4.641481  0.688500  19.95  21.0000  21.0000   
True        10047.0   6.263126  4.978278  0.305118   1.88   4.9077   9.7422   

                     age                                                       
           max     count       mean        std   min   25%   50%   75%    max  
miInSim                                                                        
False    21.00  139953.0  61.894415  15.111863  19.0  50.0  60.0  72.0  105.0  
True     18.98   10047.0  72.401911  14.734775  21.0  62.0  74.0  84.0  105.0

In [157]:
df['matchAge'] = df['baseAge']
df.loc[(df.miInSim==True),'matchAge'] = df.ageAtFirstMI

mi = df.loc[df.miInSim==True]
nonMI = df.loc[df.miInSim==False]

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

def get_matching_pairs(treated_df, non_treated_df, scaler=True):
    treated_x = treated_df.values
    non_treated_x = non_treated_df.values
    if scaler == True:
        scaler = StandardScaler()
    if scaler:
        scaler.fit(treated_x)
        treated_x = scaler.transform(treated_x)
        non_treated_x = scaler.transform(non_treated_x)

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(non_treated_x)
    distances, indices = nbrs.kneighbors(treated_x)
    return indices.reshape(indices.shape[0])

columns = ['matchAge']
matchedIndices = get_matching_pairs(mi[columns], nonMI[columns])

print(f"qalys in mi: {mi.qalysPostMI.mean():.1f} \nqalys In control: {nonMI.iloc[matchedIndices].qalysPostMI.mean():.1f} ")
print(f"\nage in mi: {mi.matchAge.mean():.1f} \nage In control: {nonMI.iloc[matchedIndices].matchAge.mean():.1f} ")

mi['age5'] = pd.qcut(mi.matchAge, 5)
mi.groupby('age5')[['qalysPostMI']].mean()

qalys in mi: 6.3 
qalys In control: 15.9 

age in mi: 66.0 
age In control: 65.4 


/Users/burke/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,qalysPostMI
age5,
"(19.999, 52.0]",8.869412
"(52.0, 62.0]",7.511304
"(62.0, 71.0]",6.590708
"(71.0, 80.0]",5.069363
"(80.0, 104.0]",3.148203


### for MI, it looks like we're slightly overestimaging by comparison to the literature, but we're in teh ballpark and we have reasonable age stratification
Thokala, P., Goodacre, S. W., Collinson, P. O., Stevens, J. W., Mills, N. L., Newby, D. E., et al. (2012). Cost-effectiveness of presentation versus delayed troponin testing for acute myocardial infarction. Heart (British Cardiac Society), 98(20), 1498–1503. http://doi.org/10.1136/heartjnl-2012-302188

In [151]:
df['matchAge'] = df['baseAge']
df.loc[(df.strokeInSim==True),'matchAge'] = df.ageAtFirstStroke

stroke = df.loc[df.strokeInSim==True]
nonStroke = df.loc[df.strokeInSim==False]

matchedIndices = get_matching_pairs(stroke[columns], nonStroke[columns])

print(f"qalys in stroke: {stroke.qalysPostStroke.mean():.1f} \nqalys In control: {nonStroke.iloc[matchedIndices].qalysPostStroke.mean():.1f} ")
print(f"\nage in stroke: {stroke.matchAge.mean():.1f} \nage In stroke: {nonStroke.iloc[matchedIndices].matchAge.mean():.1f} ")

qalys in stroke: 5.1 
qalys In control: 12.3 

age in stroke: 73.6 
age In stroke: 72.4 


### one case where it looks like we're slightly "under" what is i the published liiterature, buty we'r ein teh ballpark

Young, K. C., Benesch, C. G., & Jahromi, B. S. (2010). Cost-effectiveness of multimodal CT for evaluating acute stroke. Neurology, 75(19), 1678–1685. http://doi.org/10.1212/WNL.0b013e3181fc2838

In [152]:
df['matchAge'] = df['baseAge']
df.loc[(df.incidentDementia==True),'matchAge'] = df.ageAtFirstDementia

dementia = df.loc[df.incidentDementia==True]
nonDementia = df.loc[df.incidentDementia==False]

matchedIndices = get_matching_pairs(dementia[columns], nonDementia[columns])

print(f"qalys in dementia: {dementia.qalysPostDementia.mean():.1f} \nqalys In control: {nonDementia.iloc[matchedIndices].qalysPostDementia.mean():.1f} ")
print(f"\nage in dementia: {dementia.matchAge.mean():.1f} \nage In control: {nonDementia.iloc[matchedIndices].matchAge.mean():.1f} ")

qalys in dementia: 4.4 
qalys In control: 6.4 

age in dementia: 80.2 
age In control: 78.2 


### dementia numbers are also in teh ballpark...
Green, C., Handels, R., Gustavsson, A., Wimo, A., Winblad, B., Sköldunger, A., & Jönsson, L. (2019). Assessing cost-effectiveness of early intervention in Alzheimer's disease: An open-source modeling framework. Alzheimer's & Dementia, 15(10), 1309–1321. http://doi.org/10.1016/j.jalz.2019.05.004